### **Modelling stock price data**

##### **Model SPY yearly data**

Model SPY yearly price change data with macroeconomic features: emplyment, interest rates, house prices, consumer prices, credit etc.

While correlation does not necessarly mean causation, modelling price change and finding the feature weights can give us an idea of how the stock market moves.

Data:
1. Yahoo finance SPY price change
1. Bank if international settlements webpage [BIS](www.bis.org)

##### **Model Quarterly stock data**

We will model stock price data (log price or percentage change) using quarterly net earnings, US policy rates, property prices and employment data.

Before modelling, we will do some data validation to understand the distribution of each feature and their relationship with the dependent variable.

We will use the following models:

1. Basic multivariate regression model
2. GAM to model the non-linear relationships between independent and dependent variables
3. Bayesian model 

Stocks to model: MSFT, BAC

In [1]:
# Data manipulation
import numpy as np
import pandas as pd

In [7]:
# load data 
df_spy = pd.read_csv('./data/SPY.csv') # monthly SPY data
df_cp = pd.read_excel('./data/consumer_prices.xlsx', sheet_name='data') # consumer prices, monthly
df_pp = pd.read_excel('./data/properties_prices.xlsx', sheet_name='data') # properity prices, quarterly
df_ir = pd.read_excel('./data/interest_rates.xlsx', sheet_name='monthly') # interest rates, monthly
df_tc = pd.read_excel('./data/totcredit.xlsx', sheet_name='quarterly_data') # total credit, quarterly
df_nfp = pd.read_excel('./data/nfp_total.xlsx') # nfp employment data, monthly

In [3]:
df_spy.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-02-01,43.96875,45.12500,42.81250,44.40625,25.709480,5417600
1,1993-03-01,44.56250,45.84375,44.21875,45.18750,26.161797,3019200
2,1993-04-01,45.25000,45.25000,43.28125,44.03125,25.612766,2697200
3,1993-05-01,44.09375,45.65625,43.84375,45.21875,26.303528,1808000
4,1993-06-01,45.37500,45.81250,44.21875,45.06250,26.212654,3438000


In [29]:
df_tc["date_month"] = df_tc['Period'].apply(lambda dt: dt.replace(day=1))
df_tc.drop(['Period'], axis=1, inplace=True)
df_tc.head()

,US,Euro,date_month
0,353.722,NaN,1945-12-01
1,354.332,NaN,1946-03-01
2,350.867,NaN,1946-06-01
3,346.826,NaN,1946-09-01
4,345.682,NaN,1946-12-01


In [20]:
df_nfp_melt = pd.melt(df_nfp, id_vars=['Year'], value_vars=df_nfp.columns, var_name='month', value_name='nfp')
df_nfp_melt.head()

,Year,month,nfp
0,1980,Jan,90800.0
1,1981,Jan,91033.0
2,1982,Jan,90565.0
3,1983,Jan,88990.0
4,1984,Jan,92673.0


In [24]:
df_nfp_melt['date'] = df_nfp_melt.Year.astype('str') + "-" + df_nfp_melt.month.astype('str') 

In [27]:
df_nfp_melt['date'] = pd.to_datetime(df_nfp_melt['date'])
df_nfp_melt.drop(['Year', 'month'], axis=1, inplace=True)
df_nfp_melt.head()

,nfp,date
0,90800.0,1980-01-01
1,91033.0,1981-01-01
2,90565.0,1982-01-01
3,88990.0,1983-01-01
4,92673.0,1984-01-01
